In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('ECE 143.csv')
df.set_index(["State"], inplace=True)

# Drop outlier
df = df.drop("California") 
df = df.drop("District of Columbia") # advanced degree outlier
df.head()

# Normalize Charging Outlet and Locations to a per person basis
# The point is to be blind to a state's population
df['Charging Locations'] = df['Charging Locations'] / df["Population"]
df['Charging Outlets'] = df['Charging Outlets'] / df["Population"]

# Compute average sales and add to df
if 'average_sales' not in df:
    df.insert(5, 'average_sales', df.iloc[:,1:5].mean(axis=1).to_frame())
    df['average_sales'] = df['average_sales'].astype(int)

In [ ]:
# Linear regression

# Adjust columns names
df.rename(columns={"Avg/C": "Average temperature(Celsius)",
                   "Median Household Income\t $": "Median Household Income", 
                   "% Libertarian/ Independent Representation": "% Libertarian/Independent Representation",
                   "Avg gasoline price per gallon": "Average gasoline price per gallon",
                   "COMMUTE TIME": "Commute time",
                   "PUBLIC TRANSIT USAGE": "Public transit usage",
                   "ROAD QUALITY": "Road quality",
                   "BRIDGE QUALITY": "Bridge quality"
                   }, inplace=True)

# Normalize Average Sales to a per person basis
# Multiply by 100 to have ratio in percentage
df['average_sales/pop'] = 100 * df["average_sales"]/df["Population"]


# Correlation plot between average sales per person and continuous features
avg_sales = df['average_sales/pop']
cont_features = ['Average temperature(Celsius)', 'Commute time','Public transit usage','Road quality','Average retail price (cents/kWh)','Average gasoline price per gallon',
             '% High school graduate\nor higher',"% Bachelor's degree\nor higher",'Advanced degree','Democratic Representation','Republican Representation',
             'Median Household Income','Charging Locations',
             'Charging Outlets']
cont_cols = df.loc[:,cont_features]
cont_cols.columns = cont_features

# Plot correlation plots
fig, axs = plt.subplots(7, 2,figsize=(15,20),constrained_layout=True)
fig.suptitle("Correlation with Average EV Sales per Capita",fontsize=22)
for i in range(14):
    r = i//2
    c = i%2
    
    # get feature, normalize and plot
    col = cont_cols.iloc[:,i-1].astype(float)
    col=col.apply(lambda x : (x-col.mean())/col.std())
    axs[r, c].plot(col.iloc[:], avg_sales.iloc[:], 'bo')
    
    # set plot title, showing correlation value
    corr = avg_sales.corr(col)
    axs[r, c].set_title("{}. r: {:2f}".format(col.name,corr))
    
    # format plot
    axs[r, c].xaxis.set_visible(False)
    axs[r, c].yaxis.set_visible(False)
    x0,x1 = axs[r, c].get_xlim()
    y0,y1 = axs[r, c].get_ylim()
    axs[r, c].set_aspect(abs(x1-x0)/abs(y1-y0))
plt.savefig("plot.png")


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import statistics as stats

# Top features selected from correlation plots
X = df.loc[:,['Public transit usage','Charging Locations', 'Charging Outlets', 'Republican Representation', "% Bachelor's degree\nor higher", 'Median Household Income','Average gasoline price per gallon']]
Y = df['average_sales/pop']

# Divide data into test and training sets
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=3)

# Scale data
min_max_scaler = preprocessing.StandardScaler()
X_train_scaled = min_max_scaler.fit_transform(x_train)
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X.columns)
X_test_scaled = min_max_scaler.transform(x_test)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X.columns)


# Fit data and print coefficients
lrModel = LinearRegression()
x = X_train_scaled
y = y_train
lrModel.fit(x,y)
print("Linear Regression Training score: " + str(lrModel.score(x,y)))
print("Weights:")
for i in range(len(X.columns)):
    print("{}: {}".format(X.columns[i], lrModel.coef_[i]))
print("Bias: " + str(lrModel.intercept_))

pred = lrModel.predict(X_test_scaled)
result = pd.DataFrame({ 'true_labels': y_test, 'prediction': pred }) 
print("\nTest set performance")
print(result)
print("Test score: " + str(lrModel.score(x_test_scaled,y_test)))
print("Mean squared error: {}".format(stats.mean([(y_test[i]-pred[i])**2 for i in range(len(pred))])))


# Plot Figure
plt.figure(figsize=(20,10))
plt.plot(pred, 'rx', label='prediction')
plt.plot(y_test, 'bx', label='truth')
plt.suptitle("Test performance",fontsize=22)

In [ ]:
# Decision Tree

from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import export_graphviz
from IPython.display import Image, display

# Create Decision Tree Regressor of depth 2
tree_reg_2 = DecisionTreeRegressor(max_depth=2)
tree_reg_2.fit(x_train,y_train)
export_graphviz( tree_reg_2,
out_file="ev_tree_max_depth_2.dot", feature_names=list(x.columns))

display(Image(filename='ev_tree_2_avg_sales.png'))
print("\nTest set performance")
pred = tree_reg_2.predict(x_test)
result = pd.DataFrame({ 'true_labels': y_test, 'prediction': pred })
print(result)
plt.figure(figsize=(20,10))
plt.plot(pred, 'rx', label='prediction')
plt.plot(y_test, 'bx', label='truth')
plt.suptitle("Test performance",fontsize=22)
#plt.xticks(rotation=90)

y_train_pred = tree_reg_2.predict(x_train)
y_test_pred = tree_reg_2.predict(x_test)
print('R^2 train: %.3f, test: %.3f' % (r2_score(y_train, y_train_pred), r2_score(y_test, y_test_pred)))

print("Mean squared error: {}".format(stats.mean([(y_test[i]-pred[i])**2 for i in range(len(pred))])))


In [ ]:
# Create Decision Tree Regressor of depth 3
tree_reg_3 = DecisionTreeRegressor(max_depth=3)
tree_reg_3.fit(x_train, y_train)
export_graphviz( tree_reg_3,
out_file="ev_tree_max_depth_3.dot", feature_names=list(x.columns))

from sklearn.metrics import mean_squared_error, r2_score
display(Image(filename='ev_tree_3_avg_sales.png'))
print("\nTest set performance")
pred = tree_reg_3.predict(x_test)
result = pd.DataFrame({ 'true_labels': y_test, 'prediction': pred })
print(result)
plt.figure(figsize=(20,10))
plt.plot(pred, 'rx', label='prediction')
plt.plot(y_test, 'bx', label='truth')
plt.suptitle("Test performance",fontsize=22)

y_train_pred = tree_reg_3.predict(x_train)
y_test_pred = tree_reg_3.predict(x_test)
print('R^2 train: %.3f, test: %.3f' % (r2_score(y_train, y_train_pred), r2_score(y_test, y_test_pred)))

print("Mean squared error: {}".format(stats.mean([(y_test[i]-pred[i])**2 for i in range(len(pred))])))

In [ ]:
# Random forest 

# Create Random Forest Regressor w/ 1000 estimators and using MSE criterion
from sklearn.ensemble import RandomForestRegressor

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=3)

forest = RandomForestRegressor(n_estimators=1000, criterion='mse', random_state=1, n_jobs=-1)
forest.fit(X_train, y_train)
y_train_pred = forest.predict(X_train)
y_test_pred = forest.predict(X_test)
print('R^2 train: %.3f, test: %.3f' % (r2_score(y_train, y_train_pred), r2_score(y_test, y_test_pred)))


plt.figure(figsize=(20,10))
plt.plot(y_test_pred, 'rx', label='prediction')
plt.plot(y_test, 'bx', label='truth')
plt.suptitle("Test performance",fontsize=22)
plt.show()